# Load Combinations
*Pystra* includes the class `LoadCombination` which can be used to analyse a structural reliability problem for different load cases. This example demonstrates the use of load combination cases with *Pystra*.

This demonstration problem is adopted from Example 4 pp 190-1 from Sorensen, J.D. (2004), *Notes in Structural Reliability Theory And Risk Analysis*. The parameter values are slightly modified for demonstration.

## Import Libraries

In [1]:
import pystra as ra
import numpy as np

## Define the limit state function
The LSF to be supplied in `LoadCombination` can be any valid *Pystra* LSF. Here we show how random variables and even constants (e.g. `cg`) can be passed through as *Pystra* objects, or can be hard-coded into the LSF.

In [2]:
def lsf(R, G, Q1, Q2, cg):
    return R - (cg*G + 0.6*Q1 + 0.3*Q2)

## Define the Load and Resistance distributions
We define the distributions of the loads for the load combinations. Some loads will be time-invariant (e.g. permanent loads), while some will be combination random variables (e.g. winds, imposed loads). These combination loads require the definition of a point-in-time (`'pit'`) distribution and a maximum (`'max'`) distribtribution (typically annual maximum). Note that *Pystra* has built in distribution objects to convert between the two: `MaxParent` and `Maximum`.

For the combination loads, we define a dictionary which contains a key for each variable in the LSF, and a dictionary with the `pit` and `max` key values being the corresponding *Pystra* distribution objects.

In [3]:
# Annual max distributions
Q1max = ra.Gumbel("Q1", 1, 0.2)  # Imposed Load
Q2max = ra.Gumbel("Q2", 1, 0.4)  # Wind Load
# Parameters of inferred point-in-time parents
Q1pit = ra.Gumbel("Q1", 0.89, 0.2)  # Imposed Load
Q2pit = ra.Gumbel("Q2", 0.77, 0.4)  # Wind Load
Q_dict = {'Q1': {'max': Q1max, 'pit': Q1pit}, 
          'Q2': {'max': Q2max, 'pit': Q2pit}}
# Define any constants to be passed through
cg = ra.Constant("cg", 0.4)
## Define other random variables
Rdist = ra.Lognormal("R", 2.0, 0.15)  # Resistance
Gdist = ra.Normal("G", 1, 0.1)  # Permanent Load (Other load variable)

## Specify load combination cases
For this problem, in addition to the permanent loads, we're interested in investigating the reliabilities for several load combinations:
1. `Q1Q2_max`: Where both `Q1` and `Q2` maximum distributions are considered simultaneously, as an obvious conservative estimation.
2. `Q1_max`: `Q1` is maximum and  `Q2` is the point-in-time distribution.
3. `Q2_max`: `Q1` is the point-in-time distribution and  `Q2` is maximum.

We define a dictionary with the relevant loadcase names and corresponding variable names form the LSF.

In [4]:
loadcombinations = {'Q1Q2_max':['Q1', 'Q2'], 'Q1_max':['Q1'], 'Q2_max':['Q2']}

**Note**: The specification of this dictionary is optional. By default, `LoadCombination` assumes one combination load to act at its maximum in a load case, while the others are considered to be the point-in-time distribution.

## Instantiate `LoadCombination` object
`LoadCombination` class requires specification of combination distributions as a dictionary, while the other distributions (permanent loads and resistances) are specified as a list.

In [5]:
lc = ra.LoadCombination(lsf, Q_dict, [Rdist], [Gdist], [cg],
                          dict_comb_cases=loadcombinations)

## Analyse Load Cases

Use a list to keep track of each `Form` object outputted:

In [6]:
form = 3*[None]

### Load Case 1

In [7]:
lc_idx = 0
form[lc_idx] = lc.run_reliability_case(lcn='Q1Q2_max')
form[lc_idx].showDetailedOutput()


FORM
Pf              	 2.3296784889e-03
BetaHL          	 2.8296869486
Model Evaluations 	 49
------------------------------------------------------
Variable   	    U_star 	       X_star 	     alpha
R          	 -1.176605 	     1.826170 	 -0.415775
G          	  0.344173 	     1.034417 	 +0.121598
Q1         	  1.803368 	     1.427003 	 +0.637317
Q2         	  1.803368 	     1.854005 	 +0.637317
cg         	       --- 	     0.400000 	       ---



### Load Case 2

In [8]:
lc_idx = 1
form[lc_idx] = lc.run_reliability_case(lcn='Q1_max')
form[lc_idx].showDetailedOutput()


FORM
Pf              	 1.2027976121e-03
BetaHL          	 3.0349701148
Model Evaluations 	 49
------------------------------------------------------
Variable   	    U_star 	       X_star 	     alpha
R          	 -1.206238 	     1.822122 	 -0.397410
G          	  0.353656 	     1.035366 	 +0.116485
Q1         	  1.953325 	     1.480813 	 +0.643621
Q2         	  1.953325 	     1.731626 	 +0.643621
cg         	       --- 	     0.400000 	       ---



### Load Case 3

In [9]:
lc_idx = 2
form[lc_idx] = lc.run_reliability_case(lcn='Q2_max')
form[lc_idx].showDetailedOutput()


FORM
Pf              	 1.2381361918e-03
BetaHL          	 3.0262258803
Model Evaluations 	 49
------------------------------------------------------
Variable   	    U_star 	       X_star 	     alpha
R          	 -1.205073 	     1.822281 	 -0.398173
G          	  0.353282 	     1.035328 	 +0.116698
Q1         	  1.946925 	     1.368458 	 +0.643366
Q2         	  1.946925 	     1.956916 	 +0.643366
cg         	       --- 	     0.400000 	       ---



#### Display reliability per load combination

In [10]:
for lc,β in zip([k for k in loadcombinations.keys()], [f.beta for f in form]):
    print(f"{lc}: β = {β[0]:.3f}")

Q1Q2_max: β = 2.830
Q1_max: β = 3.035
Q2_max: β = 3.026
